# pymcfost tutorial


This notebook illustrate some of the main capabilities of pymcfost to run and explore mcfost models. We assume here that you are already familiar with the use of mcfost

We first perform basic imports.
The python package is named pymcfost to avoid confusion with mcfost itself, but we import it as mcfost for convenience.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pymcfost as mcfost

## Running a mcfost model


``mcfost.run`` performs a system call and run the mcfost binary, assuming it is available in your system path.

We make sure to delete any potential previous model.
Note that when running pymcfost from a notebook, the mcfost output is displayed in the terminal, not in the notebook.

In [ ]:
mcfost.run("/Users/cpinte/mcfost/src/ref4.0.para", delete_previous=True)

### Reading the model and plotting the temperature map and SED


We first read the model:


In [ ]:
model = mcfost.SED("./data_th/")

We can acces the parameter file values via model.P and display them:

In [ ]:
print(model.P)

 and plot the temperature structure:


In [ ]:
model.plot_T()

a "log" view makes it easier to see the temperature structure:

In [ ]:
model.plot_T(log=True)

We can also plot the SED for the 1st inclination:

In [ ]:
model.plot(0)

The SED above is a bit noisy in the mid-infrared, we can increase the number of packets by a factor 10, and re-run the model. We also use a blackbody for the star instead of a proper spectrum.

In [ ]:
import copy
P = copy.copy(model.P)
P.phot.nphot_SED *= 10     # 10 times fmore packets for the SED
P.stars[0].is_bb = False   # we do not want the star to be a black-body
P.writeto("tmp.para")
mcfost.run("tmp.para", delete_previous=True)  # We need to recompute everything because we changed the star

Let's read the new model and plot it again.

We can also plot the various contribution:
 - pink : direct stellar light
 - blue : scattered stellar light
 - red : direct thermal emission
 - green : scattered thermal emission

In [ ]:
model = mcfost.SED("./data_th/")
model.plot(0, contrib=True)
plt.ylim(5e-16,5e-12)   # we reduce the range on the y axis

## Scattered light images and polarisation maps

We can compute scattered-light images at 1micron and plot the corresponding maps.

In [ ]:
mcfost.run("/Users/cpinte/mcfost/src/ref4.0.para",options = "-img 1.0")

image_1mum = mcfost.Image("./data_1.0/")

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,4))
cbar = False
no_ylabel = False
for i in range(3):
    if i==2:
        cbar=True
    if i>0:
        no_ylabel=True
    image_1mum.plot(i, ax=axes[i], vmax=1e-15, colorbar=cbar, no_ylabel=no_ylabel)

We can also plot the corresponding polarisation maps, for instance the Qphi map with overlayed polarisation vectors:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,4))
cbar = False
no_ylabel = False
for i in range(3):
    if i>0:
        no_ylabel=True
    image_1mum.plot(i, ax=axes[i], type="Qphi", vmax=1e-15, colorbar=cbar,
                    no_ylabel=no_ylabel, pola_vector=True, nbin=15)

We can also calculate an image at sub-mm wzvelengths, for example for ALMA Band 6:

In [ ]:
mcfost.run("/Users/cpinte/mcfost/src/ref4.0.para",options = "-img 1300")
image_1mm  = mcfost.Image("./data_1300/")

In [ ]:
fig.clf()
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,4))

cbar = False
no_ylabel = False
for i in range(3):
    if i==2:
        cbar=True
    if i>0:
        no_ylabel=True
    image_1mm.plot(i, ax=axes[i], Tb=True, colorbar=cbar, no_ylabel=no_ylabel, vmax=30)

and the corresponding 12CO cubes with the `-mol` option. 
We can skip the temperature calculation with `-no_T`, as we did it before for the SED.

In [ ]:
mcfost.run("/Users/cpinte/mcfost/src/ref4.0.para",options = "-no_T -mol")

In [ ]:
mol = mcfost.Line("./data_CO/")

We can plot the integrated line profile:

In [ ]:
mol.plot_line(2)

or a given channel at velocity = 0.5km/s from systemic velocity

In [ ]:
mol.plot_map(2,v=0.5, Tb=True)

or plot the same channel after spatial convolution by a circulat beam of 0.1"

In [ ]:
mol.plot_map(2,v=0.5, bmaj=0.1, bmin=0.1, bpa=0, Tb=True)


## Running a phantom model